In [107]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
# from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn import neighbors
import pickle

In [2]:
match_results=pd.read_csv('All_stats.csv')

In [3]:
match_results

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,B365A,BWH,BWD,BWA,IWH,IWD,IWA,WHH,WHD,WHA
0,E0,15-08-2009,Aston Villa,Wigan,0,2,A,0,1,A,...,5.50,1.65,3.40,5.00,1.70,3.50,4.90,1.70,3.40,5.50
1,E0,15-08-2009,Blackburn,Man City,0,2,A,0,1,A,...,2.10,3.40,3.15,2.00,3.20,3.20,2.20,3.50,3.20,2.15
2,E0,15-08-2009,Bolton,Sunderland,0,1,A,0,1,A,...,3.25,2.15,3.15,3.15,2.20,3.20,3.20,2.30,3.20,3.20
3,E0,15-08-2009,Chelsea,Hull,2,1,H,1,1,D,...,21.00,1.17,6.00,13.00,1.20,5.50,14.00,1.17,6.50,21.00
4,E0,15-08-2009,Everton,Arsenal,1,6,A,0,3,A,...,2.30,2.95,3.15,2.25,3.10,3.10,2.30,3.20,3.20,2.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3795,E0,12-05-2019,Liverpool,Wolves,2,0,H,1,0,H,...,11.00,1.30,5.75,9.50,1.31,5.40,9.30,1.29,5.50,11.00
3796,E0,12-05-2019,Man United,Cardiff,0,2,A,0,1,A,...,11.00,1.25,6.25,11.00,1.28,6.10,9.20,1.25,6.50,10.00
3797,E0,12-05-2019,Southampton,Huddersfield,1,1,D,1,0,H,...,8.50,1.42,4.75,7.25,1.42,4.80,7.10,1.40,4.75,8.00
3798,E0,12-05-2019,Tottenham,Everton,2,2,D,1,0,H,...,3.50,2.10,3.50,3.50,2.05,3.45,3.60,2.10,3.50,3.50


In [4]:
match_results.isnull().sum()

Div         0
Date        0
HomeTeam    0
AwayTeam    0
FTHG        0
FTAG        0
FTR         0
HTHG        0
HTAG        0
HTR         0
Referee     0
HS          0
AS          0
HST         0
AST         0
HF          0
AF          0
HC          0
AC          0
HY          0
AY          0
HR          0
AR          0
B365H       0
B365D       0
B365A       0
BWH         1
BWD         1
BWA         1
IWH         1
IWD         2
IWA         1
WHH         0
WHD         0
WHA         0
dtype: int64

In [9]:
match_results.fillna(method='pad',inplace=True)

In [10]:
match_results.isnull().sum()

Div         0
Date        0
HomeTeam    0
AwayTeam    0
FTHG        0
FTAG        0
FTR         0
HTHG        0
HTAG        0
HTR         0
Referee     0
HS          0
AS          0
HST         0
AST         0
HF          0
AF          0
HC          0
AC          0
HY          0
AY          0
HR          0
AR          0
B365H       0
B365D       0
B365A       0
BWH         0
BWD         0
BWA         0
IWH         0
IWD         0
IWA         0
WHH         0
WHD         0
WHA         0
dtype: int64

In [78]:
#Checking or duplicates
match_results.duplicated(keep='first').value_counts()

False    3800
dtype: int64

In [14]:
winner=[]
for i in range (len(match_results['HomeTeam'])):
    if match_results['FTHG'][i] > match_results['FTAG'][i]:
        winner.append(match_results['HomeTeam'][i])
    elif match_results['FTHG'][i] < match_results ['FTHG'][i]:
        winner.append(match_results['AwayTeam'][i])
    else:
        winner.append('Draw')
match_results['Winner'] = winner

In [12]:
#Team with most away goals
match_results['FTAG'].groupby(match_results['AwayTeam']).sum().sort_values(ascending=False).head()

AwayTeam
Man City      349
Arsenal       319
Man United    309
Tottenham     308
Liverpool     307
Name: FTAG, dtype: int64

In [13]:
#Team with most home goals
match_results['FTHG'].groupby(match_results['HomeTeam']).sum().sort_values(ascending=False).head()

HomeTeam
Man City      480
Chelsea       424
Arsenal       410
Liverpool     398
Man United    392
Name: FTHG, dtype: int64

In [43]:

match_results['AwayTeam'].unique()

array(['Wigan', 'Man City', 'Sunderland', 'Hull', 'Arsenal', 'Fulham',
       'Burnley', 'West Ham', 'Birmingham', 'Liverpool', 'Chelsea',
       'Wolves', 'Portsmouth', 'Man United', 'Tottenham', 'Stoke',
       'Bolton', 'Blackburn', 'Everton', 'Aston Villa', 'West Brom',
       'Blackpool', 'Newcastle', 'Norwich', 'Swansea', 'QPR',
       'Southampton', 'Reading', 'Cardiff', 'Crystal Palace', 'Leicester',
       'Watford', 'Bournemouth', 'Middlesbrough', 'Huddersfield',
       'Brighton'], dtype=object)

In [84]:
#To get total number of shots at home and away for a team
def total_shots(team_name):
    homeshots=0
    awayshots=0
    
    if team_name in match_results['HomeTeam'].unique():
        for i in range(len(match_results['HomeTeam'])):
            if team_name==match_results['HomeTeam'][i]:
                homeshots+=match_results['HS'][i]
            elif team_name==match_results['AwayTeam'][i]:
                awayshots+=match_results['AS'][i]
        print('Team:' ,team_name, "\nHomeShots:", homeshots, "\nAwayShots:",awayshots)
    else:
        print('Check the team name')
    

In [90]:
total_shots('Tottenham')

Team: Tottenham 
HomeShots: 3366 
AwayShots: 2639


In [91]:
#Checking corr to get required features
#Corr with HomeGoals
match_results.corr()['FTHG'].sort_values()

IWH     -0.283864
BWH     -0.280853
WHH     -0.277975
B365H   -0.274716
AS      -0.152125
HY      -0.111800
AST     -0.100444
FTAG    -0.084583
HR      -0.080648
AC      -0.080303
HTAG    -0.059961
HF      -0.059840
AF      -0.045542
AY      -0.011191
HC       0.057204
AR       0.072354
WHD      0.255717
IWD      0.258023
BWD      0.261387
B365D    0.262343
HS       0.288352
B365A    0.355840
BWA      0.357870
WHA      0.358323
IWA      0.368409
HST      0.446156
HTHG     0.684379
FTHG     1.000000
Name: FTHG, dtype: float64

In [92]:
#Corr with AwayGoals
match_results.corr()['FTAG'].sort_values()


BWA     -0.264203
IWA     -0.261981
B365A   -0.258997
WHA     -0.254824
HS      -0.129333
IWD     -0.114267
BWD     -0.112715
B365D   -0.112542
WHD     -0.107252
HST     -0.105015
HC      -0.089446
FTHG    -0.084583
HTHG    -0.076795
AR      -0.071213
AF      -0.035226
AY      -0.002574
HF       0.025352
AC       0.050975
HR       0.120085
HY       0.120465
WHH      0.314067
AS       0.315120
BWH      0.318378
B365H    0.318913
IWH      0.320778
AST      0.447233
HTAG     0.688853
FTAG     1.000000
Name: FTAG, dtype: float64

In [98]:
#Taking valid features into consideration
PL_matchresults=match_results[['HomeTeam','AwayTeam','FTHG','FTAG','HST','AST','B365H','B365D','B365A']]

In [99]:
PL_matchresults

,HomeTeam,AwayTeam,FTHG,FTAG,HST,AST,B365H,B365D,B365A
0,Aston Villa,Wigan,0,2,5,7,1.67,3.60,5.50
1,Blackburn,Man City,0,2,9,5,3.60,3.25,2.10
2,Bolton,Sunderland,0,1,3,13,2.25,3.25,3.25
3,Chelsea,Hull,2,1,12,3,1.17,6.50,21.00
4,Everton,Arsenal,1,6,5,9,3.20,3.25,2.30
...,...,...,...,...,...,...,...,...,...
3795,Liverpool,Wolves,2,0,5,2,1.30,6.00,11.00
3796,Man United,Cardiff,0,2,10,4,1.28,6.50,11.00
3797,Southampton,Huddersfield,1,1,3,3,1.44,4.75,8.50
3798,Tottenham,Everton,2,2,3,9,2.20,3.50,3.50


In [264]:
#Regular Premier League Teams
PremierLeague_teams=['Arsenal','Aston Villa','Bournemouth','Brighton','Burnley',
                    'Chelsea','Crystal Palace','Everton','Fulham',
                    'Huddersfield','Leicester','Liverpool','Man City','Man United',
                    'Newcastle','Southampton','Swansea','Tottenham','Watford','West Ham',
                     'Wolves']

In [265]:
#Matches played by Regular Teams
PL_matchresults=PL_matchresults[PL_matchresults['HomeTeam'].isin(PremierLeague_teams) & PL_matchresults['AwayTeam'].isin(PremierLeague_teams)]


In [266]:
PL_matchresults

,HomeTeam,AwayTeam,HomeTeam_ShotsonTarget,AwayTeam_ShotsonTarget,B365H,B365D,B365A,HomeGoals,AwayGoals
0,Everton,Arsenal,5,9,3.20,3.25,2.30,1,6
1,Wolves,West Ham,11,13,2.63,3.30,2.70,0,2
2,Tottenham,Liverpool,11,3,3.20,3.25,2.30,2,1
3,Burnley,Man United,2,9,9.50,5.00,1.33,1,0
4,Man City,Wolves,11,2,1.33,4.50,12.00,1,0
...,...,...,...,...,...,...,...,...,...
2037,Leicester,Chelsea,3,4,2.40,3.75,2.90,0,0
2038,Liverpool,Wolves,5,2,1.30,6.00,11.00,2,0
2039,Southampton,Huddersfield,3,3,1.44,4.75,8.50,1,1
2040,Tottenham,Everton,3,9,2.20,3.50,3.50,2,2


In [267]:
PL_matchresults.reset_index(inplace=True,drop=True)

In [268]:
PL_matchresults

,HomeTeam,AwayTeam,HomeTeam_ShotsonTarget,AwayTeam_ShotsonTarget,B365H,B365D,B365A,HomeGoals,AwayGoals
0,Everton,Arsenal,5,9,3.20,3.25,2.30,1,6
1,Wolves,West Ham,11,13,2.63,3.30,2.70,0,2
2,Tottenham,Liverpool,11,3,3.20,3.25,2.30,2,1
3,Burnley,Man United,2,9,9.50,5.00,1.33,1,0
4,Man City,Wolves,11,2,1.33,4.50,12.00,1,0
...,...,...,...,...,...,...,...,...,...
2037,Leicester,Chelsea,3,4,2.40,3.75,2.90,0,0
2038,Liverpool,Wolves,5,2,1.30,6.00,11.00,2,0
2039,Southampton,Huddersfield,3,3,1.44,4.75,8.50,1,1
2040,Tottenham,Everton,3,9,2.20,3.50,3.50,2,2


In [109]:
#Rearranging columns
PL_matchresults=PL_matchresults[['HomeTeam','AwayTeam','HST','AST','B365H','B365D','B365A','FTHG','FTAG']]

In [269]:
PL_matchresults

,HomeTeam,AwayTeam,HomeTeam_ShotsonTarget,AwayTeam_ShotsonTarget,B365H,B365D,B365A,HomeGoals,AwayGoals
0,Everton,Arsenal,5,9,3.20,3.25,2.30,1,6
1,Wolves,West Ham,11,13,2.63,3.30,2.70,0,2
2,Tottenham,Liverpool,11,3,3.20,3.25,2.30,2,1
3,Burnley,Man United,2,9,9.50,5.00,1.33,1,0
4,Man City,Wolves,11,2,1.33,4.50,12.00,1,0
...,...,...,...,...,...,...,...,...,...
2037,Leicester,Chelsea,3,4,2.40,3.75,2.90,0,0
2038,Liverpool,Wolves,5,2,1.30,6.00,11.00,2,0
2039,Southampton,Huddersfield,3,3,1.44,4.75,8.50,1,1
2040,Tottenham,Everton,3,9,2.20,3.50,3.50,2,2


In [115]:
#Renaming columns
PL_matchresults.rename(columns={'HST':'HomeTeam_ShotsonTarget','AST':'AwayTeam_ShotsonTarget','FTHG':'HomeGoals','FTAG':'AwayGoals'},inplace=True)

In [270]:
X=PL_matchresults.iloc[:,0:7]

In [271]:
y=PL_matchresults.iloc[:,7:9]

In [272]:
X=pd.get_dummies(X,columns=['HomeTeam','AwayTeam'])

In [273]:
X

,HomeTeam_ShotsonTarget,AwayTeam_ShotsonTarget,B365H,B365D,B365A,HomeTeam_Arsenal,HomeTeam_Aston Villa,HomeTeam_Bournemouth,HomeTeam_Brighton,HomeTeam_Burnley,...,AwayTeam_Liverpool,AwayTeam_Man City,AwayTeam_Man United,AwayTeam_Newcastle,AwayTeam_Southampton,AwayTeam_Swansea,AwayTeam_Tottenham,AwayTeam_Watford,AwayTeam_West Ham,AwayTeam_Wolves
0,5,9,3.20,3.25,2.30,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,11,13,2.63,3.30,2.70,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,11,3,3.20,3.25,2.30,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,2,9,9.50,5.00,1.33,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
4,11,2,1.33,4.50,12.00,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2037,3,4,2.40,3.75,2.90,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2038,5,2,1.30,6.00,11.00,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2039,3,3,1.44,4.75,8.50,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2040,3,9,2.20,3.50,3.50,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [274]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [275]:

rf_regressor=RandomForestRegressor(n_estimators=80,random_state=50)

In [276]:

rf_regressor.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=80, n_jobs=None, oob_score=False,
                      random_state=50, verbose=0, warm_start=False)

In [277]:
# predictions=knn_regressor.predict(X_test)
predictions=rf_regressor.predict(X_test)

In [278]:
# predictions=predictions.round().astype(int)

In [279]:
# predictions[120]

array([1, 1])

In [280]:
classification_report(y_test.iloc[:,0],predictions[:,0])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'              precision    recall  f1-score   support\n\n           0       0.69      0.26      0.38        97\n           1       0.45      0.52      0.48       133\n           2       0.32      0.51      0.39       103\n           3       0.22      0.21      0.22        53\n           4       0.33      0.06      0.10        17\n           5       0.00      0.00      0.00         5\n           6       0.00      0.00      0.00         1\n\n    accuracy                           0.39       409\n   macro avg       0.29      0.22      0.22       409\nweighted avg       0.43      0.39      0.38       409\n'

In [281]:
classification_report(y_test.iloc[:,1],predictions[:,1])

'              precision    recall  f1-score   support\n\n           0       0.58      0.23      0.33       136\n           1       0.36      0.60      0.45       134\n           2       0.30      0.41      0.35        86\n           3       0.29      0.11      0.15        38\n           4       0.00      0.00      0.00        11\n           5       0.00      0.00      0.00         2\n           6       0.00      0.00      0.00         2\n\n    accuracy                           0.37       409\n   macro avg       0.22      0.19      0.18       409\nweighted avg       0.40      0.37      0.34       409\n'

In [282]:
filename='Premier-League-Prediction-model.pkl'
pickle.dump(rf_regressor, open(filename, 'wb'))

In [294]:
team_1=['Burnley']
team_2=['Man City']
PL_matchresults[PL_matchresults['HomeTeam'].isin(team_1) & PL_matchresults['AwayTeam'].isin(team_2)]['B365A'].mean()



1.378

In [295]:
PL_matchresults[PL_matchresults['HomeTeam'].isin(team_1) & PL_matchresults['AwayTeam'].isin(team_2)]['B365H'].mean()

11.5